In [1]:
import numpy as np
import tensorflow as tf
import autograd
from autograd import numpy as anp

import scipy as sp
from scipy import optimize

from normal_example_lib import \
    normal_lp_suff, normal_lp, flatten_par, fold_par, get_objectives

In [2]:
from matplotlib import pyplot as plt
%matplotlib inline

In [3]:
dim = 3
num_obs = 10000

mu_true = np.arange(dim, dtype=np.float64)
sd_true = 0.5
tau_true = 1 / sd_true ** 2
print(tau_true)

x = np.random.normal(loc=mu_true, scale=sd_true, size=(num_obs, dim))
xsum = np.sum(x, axis=0)
x2sum = x.T @ x

muhat = xsum / num_obs
covhat = x2sum / num_obs - np.outer(muhat, muhat) 
tauhat = 1 / np.mean(np.diag(covhat))
print(tauhat)


4.0
4.031689876244987


In [4]:
normal_lp_suff(
    mu=mu_true,
    mu2=np.outer(mu_true, mu_true),
    tau=tau_true,
    log_tau=np.log(tau_true),
    xsum=xsum,
    x2sum=x2sum,
    num_obs=num_obs)

<tf.Tensor: shape=(), dtype=float64, numpy=5912.032874155855>

In [5]:

data = { 'xsum': xsum, 'x2sum': x2sum, 'num_obs': num_obs}
par = { 'mu': mu_true, 'tau': tau_true }
normal_lp(par, data)

<tf.Tensor: shape=(), dtype=float64, numpy=5912.032874155855>

In [6]:
par_flat = flatten_par(par)
par_fold = fold_par(par_flat)
print(par['tau'], par_fold['tau'])
normal_lp(par_fold, data)

4.0 tf.Tensor(4.0, shape=(), dtype=float64)


<tf.Tensor: shape=(), dtype=float64, numpy=5912.032874155855>

In [7]:
normal_objective, normal_objective_grad, normal_objective_hessian = get_objectives(data)

lp = normal_objective(par_flat)
grad = normal_objective_grad(par_flat)
hess = normal_objective_hessian(par_flat)

lp, grad, hess

-0.5912032874155855
-0.5912032874155855
-0.5912032874155855


(-0.5912032874155855,
 array([ 0.0074571 ,  0.01279885, -0.00299633, -0.01176175]),
 array([[ 4.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          7.45710490e-03],
        [ 0.00000000e+00,  4.00000000e+00,  0.00000000e+00,
          1.27988451e-02],
        [ 0.00000000e+00,  0.00000000e+00,  4.00000000e+00,
         -2.99633086e-03],
        [ 7.45710490e-03,  1.27988451e-02, -2.99633086e-03,
          1.48823825e+00]]))

In [8]:
par_flat = np.random.random(dim  + 1)
lp = normal_objective(tf.convert_to_tensor(par_flat))
grad = normal_objective_grad(tf.convert_to_tensor(par_flat))
hess = normal_objective_hessian(par_flat)

lp, grad, hess

3.098163552483872
3.098163552483872
3.098163552483872


(3.098163552483872,
 array([ 0.57358346, -1.28924584, -3.65494473,  2.79032299]),
 array([[ 2.21393829,  0.        ,  0.        ,  0.57358346],
        [ 0.        ,  2.21393829,  0.        , -1.28924584],
        [ 0.        ,  0.        ,  2.21393829, -3.65494473],
        [ 0.57358346, -1.28924584, -3.65494473,  4.29032299]]))

In [9]:
opt_result = sp.optimize.minimize(
    x0=np.zeros(dim + 1),
    fun=normal_objective,
    jac=normal_objective_grad,
    hess=normal_objective_hessian,
    method="trust-exact")

2.870358017715933
2.870358017715933
2.870358017715933
1.0765031660029623
1.0765031660029623
1.0765031660029623
-0.28368787776332793
-0.28368787776332793
-0.28368787776332793
-0.4984050723023785
-0.4984050723023785
-0.4984050723023785
-0.588313243808569
-0.588313243808569
-0.588313243808569
-0.5912732908021225
-0.5912732908021225
-0.5912732908021225
-0.5912784182781725
-0.5912784182781725
-0.5912784182781725


In [10]:
lp = normal_objective(opt_result.x)
grad = normal_objective_grad(opt_result.x)
hess = normal_objective_hessian(opt_result.x)
grad, hess

-0.5912784182781725
-0.5912784182781725
-0.5912784182781725


(array([ 6.98816700e-09, -3.73646715e-06, -7.49973020e-06,  5.11490014e-06]),
 array([[ 4.03170362e+00,  0.00000000e+00,  0.00000000e+00,
          6.98816700e-09],
        [ 0.00000000e+00,  4.03170362e+00,  0.00000000e+00,
         -3.73646715e-06],
        [ 0.00000000e+00,  0.00000000e+00,  4.03170362e+00,
         -7.49973020e-06],
        [ 6.98816700e-09, -3.73646715e-06, -7.49973020e-06,
          1.50000511e+00]]))

In [11]:
par_opt = fold_par(opt_result.x)
print(par_opt['mu'].numpy(), muhat)
print(par_opt['tau'].numpy(), tauhat, tau_true)

[-1.86427449e-03  9.96799362e-01  2.00074722e+00] [-1.86427623e-03  9.96800289e-01  2.00074908e+00]
4.031703624015668 4.031689876244987 4.0


In [24]:
@tf.function
def normal_lp_fun(par_flat):
    return -1 * normal_lp(fold_par(par_flat), data)

In [30]:
print(normal_lp_fun(opt_result.x))
f = normal_lp_fun.get_concrete_function(opt_result.x)
f(tf.convert_to_tensor(opt_result.x))

tf.Tensor(-5912.784182781725, shape=(), dtype=float64)


<tf.Tensor: shape=(), dtype=float64, numpy=-5912.784182781725>